In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta 
import matplotlib.pyplot as plt
import statsmodels as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
df1 = pd.read_csv('../../data/train_validation_marker.csv')

In [3]:
df2 = df1[df1['Encoded_SKU_ID'].isin([27, 28, 65, 83, 203, 223, 240, 288, 308, 418, 439, 451, 478])]

In [4]:
df3 = df1[df1['validation_clean'] == True]

In [5]:
df3.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,max_SALES_DATE,delta_days,validation,validation_clean
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,1,2022-07-31,1767,False,True
1,2,2017-11-24,CAMERA BATTERIES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,39.99,?,?,Fully-Stocked,18,2022-07-31,1710,False,True
2,3,2019-11-28,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,?,46.98,Constrained,7,2022-07-31,976,False,True
4,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,7,2022-07-31,1373,False,True
5,5,2018-12-18,SO CAR ACCESSORIES,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,16.99,?,?,Fully-Stocked,5,2022-07-31,1321,False,True


In [6]:
df3.Encoded_SKU_ID.nunique()

545

In [7]:
rst_sku_df = pd.read_excel('../../results/rmse_all_preds_jan_17.xlsx')

In [8]:
rst_sku_df.Encoded_SKU_ID

0       82
1      568
2      317
3       95
4      340
      ... 
540    558
541    559
542    560
543    566
544    571
Name: Encoded_SKU_ID, Length: 545, dtype: int64

In [9]:
hw_sku = rst_sku_df[rst_sku_df['min_period']=='rmse_du_holt']

In [10]:
sku = list(hw_sku.Encoded_SKU_ID)+[76, 147, 157, 358, 413, 485, 569]

In [11]:
len(sku)

41

In [12]:
test = pd.read_excel('../../data/Validation_Data.xlsx')

In [13]:
test.Encoded_SKU_ID.nunique()

539

In [14]:
set(test.Encoded_SKU_ID) - set(rst_sku_df.Encoded_SKU_ID)

{76, 147, 157, 358, 413, 485, 569}

In [15]:
test[test.Encoded_SKU_ID.isin([27, 28, 65, 83, 203, 223, 240, 288, 308, 418, 439, 451, 478])]

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,Forecasted Units


In [16]:
def fit_es(sku,df1,test,seasonal_periods=62):
    y_pred = pd.DataFrame()
    y_actual = pd.DataFrame()

    for i in sku:
        print( i)
    # clean train data on item i
        train = df1[df1['Encoded_SKU_ID']==i][['Encoded_SKU_ID','SALES_DATE','DAILY_UNITS']]
        train['SALES_DATE'] = pd.to_datetime(train['SALES_DATE'])
        train.set_index('SALES_DATE', inplace = True)
    
        val = test[test['Encoded_SKU_ID']==i][['Encoded_SKU_ID','SALES_DATE','DAILY_UNITS']]
        val['SALES_DATE'] = pd.to_datetime(val['SALES_DATE'])
        val.set_index('SALES_DATE', inplace = True)
    
    # train the HW model with auto train & make the forecast
        model = ExponentialSmoothing(train['DAILY_UNITS'],trend='add',seasonal='add',seasonal_periods=seasonal_periods).fit()
        forecast = model.forecast(7)
        print(forecast)
    
    #Construct the predicted dataframe
        SALES_DATE = pd.date_range(start='08/01/2022', end='08/07/2022')
        prediction = pd.DataFrame(forecast,columns=['predicted']).reset_index(drop=True).set_index(SALES_DATE)
        print(prediction.index)
        prediction['Encoded_SKU_ID'] = val['Encoded_SKU_ID']
        prediction.reset_index(inplace = True)
        y_pred = pd.concat([y_pred, prediction], axis = 0)
    
    #Construct the actual dataframe
        val.reset_index(inplace = True)
        val.rename(columns={'DAILY_UNITS':'actual'}, inplace = True)
        y_actual = pd.concat([y_actual, val], axis = 0)
        
        
    return(y_pred,y_actual)

In [17]:
holt_rst, holt_actual = fit_es(sku, df1, test)

365


/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has n

1218    1.598060
1219    1.334868
1220    1.755911
1221    1.440121
1222    1.492759
1223    1.387498
1224    1.913823
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
190


/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


1891    1.670673
1892    1.836632
1893    2.035384
1894    1.537414
1895    1.734490
1896    1.536760
1897    1.701242
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
248


...

/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


1276    16.560530
1277    19.797115
1278    30.818005
1279    13.349868
1280    21.696092
1281    22.102015
1282    18.817382
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
413


/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction 

937    3.603303
938    3.039835
939    3.959853
940    5.141440
941    4.304999
942    3.448650
943    2.580019
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
485
1233    0.908757
1234    0.700219
1235    0.997280
1236    1.447328
1237    1.409739
1238    1.418128
1239    1.678635
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
569
701    2.883637
702    2.065078
703    1.519381
704    2.246047
705    1.791683
706    1.430173
707    1.793841
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')


/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [18]:
holt_rst.rename({'index':'SALES_DATE'},axis=1, inplace= True)

In [19]:
holt_rst.shape

(287, 3)

In [34]:
287/7

41.0

In [35]:
import sys
sys.path.append("/Users/yibeihu/bestbuy_project/")
from src.utils import *

In [36]:
rmse(holt_rst, holt_actual)

8.219827278591737

In [37]:
holt_rst.to_csv('../../results/holt_rst_final_02.csv')

In [24]:
df1[(df1['Encoded_SKU_ID']==1) & (df1['SALES_DATE']=='2022-07-31')]

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,max_SALES_DATE,delta_days,validation,validation_clean
411277,1,2022-07-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,69.95,Fully-Stocked,1,2022-07-31,0,True,True


In [25]:
y_actual = test[['Encoded_SKU_ID','SALES_DATE', 'DAILY_UNITS']]

In [26]:
y_actual.shape

(3773, 3)

In [27]:
forecast = df1[df1['SALES_DATE'] == '2022-07-31'][['Encoded_SKU_ID','SALES_DATE','DAILY_UNITS']]

In [28]:
forecast.shape

(556, 3)

In [29]:
y_pred = y_actual.merge(forecast, on=['Encoded_SKU_ID'], how='left')

In [30]:
y_pred.head()

,Encoded_SKU_ID,SALES_DATE_x,DAILY_UNITS_x,SALES_DATE_y,DAILY_UNITS_y
0,23,2022-08-06,3,2022-07-31,1
1,100,2022-08-02,2,2022-07-31,2
2,121,2022-08-06,6,2022-07-31,0
3,207,2022-08-03,0,2022-07-31,1
4,416,2022-08-05,9,2022-07-31,11


In [31]:
y_pred.isna().sum()

Encoded_SKU_ID    0
SALES_DATE_x      0
DAILY_UNITS_x     0
SALES_DATE_y      0
DAILY_UNITS_y     0
dtype: int64

In [32]:
y_pred_null = y_pred[['Encoded_SKU_ID','SALES_DATE_x','DAILY_UNITS_y']].rename(columns={'SALES_DATE_x':'SALES_DATE','DAILY_UNITS_y':'predicted'})

In [33]:
y_pred_null.to

AttributeError: 'DataFrame' object has no attribute 'to'

In [ ]:
y_act_null = y_actual.rename(columns={'DAILY_UNITS':'actual'})

In [ ]:
y_act_null

In [ ]:
rmse(y_pred_null,y_act_null)

In [ ]:
y_pred_null.to_csv('../../results/null_model_results_final.csv')